In [ ]:
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from mcts_inference.constraints import Constraint

from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import hamming_loss


n_samples = 10000
n_features = 128
n_classes = 32
n_labels = 2
random_state = 0

X, Y = make_multilabel_classification(
    n_samples = n_samples,
    n_features = n_features,
    n_classes = n_classes,
    n_labels = n_labels,
    random_state = random_state)

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

solver = "liblinear"
base = LogisticRegression(solver=solver)
chain = ClassifierChain(base)

chain = chain.fit(X_train, Y_train)

In [ ]:
import matplotlib.pyplot as plt
from mcts_inference.mcts import MCTS
from mcts_inference.mcts_config import MCTSConfig
from mcts_inference.brute_force import brute_force

def avg_losses(Ns, config, chain, algo= MCTS, loss_function = hamming_loss, M: int = 100, avg: int = 10):
    losses = []

    print("> n:",end="")
    for n in Ns:
        print(n, end=", ")
        constraint = Constraint(max_iter=True, n_iter=n)
        config.replace_constraint(constraint)

        avg_loss = 0
        for _ in range(avg):
            out = algo(X_test[:M], chain, config)
            avg_loss += loss_function(out, Y_test[:M])

        losses.append(avg_loss/avg)

    return losses

def plot_losses(Ns, config, algo, chain, bf: bool = True, M: int = 100, label: str = "", title: str = "", loss_function=hamming_loss):
    fig = plt.figure(figsize=(10, 5))
    
    losses = avg_losses(Ns=Ns, config=config, algo=algo, chain=chain, M=M, loss_function=loss_function)
    chain_losses = loss_function(chain.predict(X_test[:M]), Y_test[:M])

    plt.plot(Ns, losses, label=label, marker="o", markersize=3)
    plt.plot(Ns, [chain_losses]*len(Ns), label="Chain", marker="o", markersize=1)

    if bf:
        title += " vs Brute Force"
        con = MCTSConfig(n_classes, constraint=Constraint(time=True, d_time=2), parallel=True)
        brute_force_losses = loss_function(brute_force(X_test[:M], chain, config=con), Y_test[:M])
        plt.plot(Ns, [brute_force_losses]*len(Ns), label="Brute Force", marker="o", markersize=1)

    plt.xlabel("Number of Iterations")
    plt.ylabel(loss_function.__qualname__ + " Loss")

    plt.title(title)
    plt.legend()
    plt.show();

In [ ]:
from mcts_inference.mc import MC
from mcts_inference.constraints import Constraint
from mcts_inference.mcts_config import MonteCarloConfig

Ns = [n for n in range(5, 206, 20)]
config = MonteCarloConfig(n_classes, constraint=Constraint(max_iter=True, n_iter=2))
plot_losses(Ns, config, algo=MC, chain=chain, bf = False, label="MC", title="MC vs Chain", M=100, loss_function=hamming_loss)